In [1]:
import datetime, time
import numpy as np
import pandas as pd
from scipy import stats
import os
import subprocess

In [2]:

#days = '10days'
#days = '90days'
days = 'year'

sp = np.genfromtxt(os.path.join('./sims3', 'constituents.csv'), delimiter=',', dtype=None)


In [3]:
tickers = sp[:,0]
names = sp[:,1]
industries = sp[:,2]

samples = tickers.size
states = []
for i in range(0,10):
    states.append(np.zeros(samples))

In [4]:

ticker_sub = tickers[:samples]
names_sub = names[:samples]
industry_sub = industries[:samples]

companies_sub = np.vstack((ticker_sub,industry_sub))

In [5]:
#names

In [6]:
n=0
e=0
error_companies = []
final_companies = []
final_companies_name = []
final_companies_ind = []
pd.options.display.float_format = '{:,.4f}'.format

for j in xrange(tickers.size):
    try:
        t = tickers[j]
        name = names[j]
        i = industries[j]
        fname = t +"_"+ i + "_" + str(days) + "_kdes_adv.csv"
        dfrtmp = pd.read_csv(os.path.join('./sims3', fname))
        for s in range(0,9):
            states[s][n] = dfrtmp.at[s,'D']
        n=n+1
        final_companies.append(t)
        final_companies_name.append(name)
        final_companies_ind.append(i)
    except:
        e = e+1
        #print (t,i)
        error_companies.append(t)
    
print (e)
print (n)
ec = np.array((error_companies))
fc1 = np.array((final_companies))
fc2 = np.array((final_companies_name))
fc3 = np.array((final_companies_ind))
fc = np.vstack((fc1,fc2,fc3))
fc=fc.T
print fc

np.savetxt('sp_list.csv',fc,delimiter=',',fmt="%s")

82
422
[['MMM' '3M Company' 'Industrials']
 ['ABT' 'Abbott Laboratories' 'Health Care']
 ['ACN' 'Accenture plc' 'Information Technology']
 ..., 
 ['YUM' 'Yum! Brands Inc' 'Consumer Discretionary']
 ['ZBH' 'Zimmer Biomet Holdings' 'Health Care']
 ['ZION' 'Zions Bancorp' 'Financials']]


In [ ]:
for s in range(10):
    fname = company+"_"+str(s+1)+"_states_model_adv.pkl"
    filename.append(os.path.join('./sims3', fname))

In [ ]:
dfr = pd.DataFrame({  #"MLS", "Rand","Vtr","Psd","Lls","Mlns
                    '1 state' : states[0],
                    '3 states' : states[2],
                    '5 states' : states[4],
                    '7 states' : states[6],
                    '9 states' : states[8],
                    #'6 states' : states[5], 
                    #'7 states' : states[6],
                    #'8 states' : states[7],
                    #'9 states' : states[8], 
                     })

In [ ]:
dfr.tail(3)

In [ ]:
dfr.describe()[:3]

In [ ]:
d1 = dfr['1 state']
d2 = dfr['3 states']
d3 = dfr['5 states']
d4 = dfr['7 states']
d5 = dfr['9 states']


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

%matplotlib inline
plt.figure(figsize=(13, 7))
data = [d1,d1_alone,d3,d3_alone,d5,d5_alone]
box = plt.boxplot(data, 0, '',widths=0.75, patch_artist=True);
plt.xticks([1,2,3,4,5,6], ['1 state + Market', '1 state','5 state + Market', '5 state',
                         '9 state + Market', '9 state']);

colors = ['lightgreen', 'lightblue', 'lightgreen', 'lightblue', 'lightgreen', 'lightblue']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    
plt.savefig('boxplot_states_plus_market.png', bbox_inches='tight')

In [ ]:
#z_stat, p_val = stats.ranksums(d1,d2)

In [ ]:
#print (p_val)

In [ ]:
p_val1 = stats.wilcoxon(d1,d1_alone, zero_method='wilcox', correction=False)[1]
p_val2 = stats.wilcoxon(d2,d2_alone, zero_method='wilcox', correction=False)[1]
p_val3 = stats.wilcoxon(d3,d3_alone, zero_method='wilcox', correction=False)[1]
p_val4 = stats.wilcoxon(d4,d4_alone, zero_method='wilcox', correction=False)[1]
p_val5 = stats.wilcoxon(d5,d5_alone, zero_method='wilcox', correction=False)[1]


In [ ]:
print("P-value 1 ", p_val1) # Used because data is paired ! :D
print("P-value 2 ", p_val2)
print("P-value 3 ", p_val3)
print("P-value 4 ", p_val4)
print("P-value 5 ", p_val5)


In [ ]:
filename = "States_Table_Comparison_" + days + '_adv.tex'

template = r'''\documentclass[preview]{{standalone}}
\usepackage{{booktabs}}
\begin{{document}}
{}
\end{{document}}
'''

with open(filename, 'wb') as f:
    f.write(template.format(dfr.describe()[:3].to_latex()))

subprocess.call(['pdflatex', filename])

In [ ]:
dfr_pvalues =  pd.DataFrame({       
                    '': pd.Categorical(["1 state vs."]),
                    '5 states':p_val1,
                    '9 states':p_val2,
                    
                     }, index=['p-value'])
#'D':np.array([0] * 2,dtype='int32'),

In [ ]:
dfr_pvalues

In [ ]:
filename = "States_Pvalues_Comparison_" + days + '_adv.tex'

template = r'''\documentclass[preview]{{standalone}}
\usepackage{{booktabs}}
\begin{{document}}
{}
\end{{document}}
'''

with open(filename, 'wb') as f:
    f.write(template.format(dfr_pvalues.to_latex()))

subprocess.call(['pdflatex', filename])